#  Neuralne mreže

## Implementacija MLP u TensorFlow

In [66]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
%matplotlib inline

Proveravamo sta se sve nalazi u train.csv
Dropujem kolone koje nemaju previse smisla da se koriste ili koje nemaju dovoljno podataka da se koriste

In [67]:
train = pd.read_csv('train.csv')
train.info()
train.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)
train.info()
train_x = train.drop('Survived', axis=1, inplace=False)
train_y = train['Survived']
train_x.info()
train_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-

In [68]:
test = pd.read_csv('test.csv')
test.info()
test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)
test.info()
test_x = test.drop('Survived', axis=1, inplace=False)
test_y = test['Survived']
test_x.info()
test_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
 11  Survived     418 non-null    int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-

Proveravam koliko ima nepopunjenih polja gde vidimo da je Age potrbno urediti to.

In [69]:
freq = train_x.count()
freq_df = pd.DataFrame([freq],columns=train_x.columns)
print(freq_df.head())

   Pclass  Sex  Age  SibSp  Parch  Fare  Embarked
0     891  891  714    891    891   891       889


Popunicemo Age sa medijanom iz pomoc StandardScalera da bi vrednosti bile izmedju 0 i 1 sto mi je kasnije laske za obradu

In [70]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")), 
    ('std_scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot', OneHotEncoder())
])
num_attr = ['Age', 'SibSp', 'Parch', 'Fare']
cat_attr = ['Pclass', 'Sex', 'Embarked']

full_pipeline = ColumnTransformer([("num", num_pipeline, num_attr), 
                                   ("cat", cat_pipeline, cat_attr)])
train_x = full_pipeline.fit_transform(train_x)
train_x.shape

num_pipeline_test = Pipeline([
    ('imputer', SimpleImputer(strategy="median")), 
    ('std_scaler', StandardScaler())
])
cat_pipeline_test = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot', OneHotEncoder())
])
num_attr_test = ['Age', 'SibSp', 'Parch', 'Fare']
cat_attr_test = ['Pclass', 'Sex', 'Embarked']

full_pipeline_test = ColumnTransformer([("num", num_pipeline_test, num_attr_test), 
                                   ("cat", cat_pipeline_test, cat_attr_test)])
test_x = full_pipeline_test.fit_transform(test_x)

train_x.shape

(891, 12)

In [71]:
test_x.shape

(418, 12)

In [72]:
nb_train = len(train_y)
nb_test = len(test_y)

# Parametri mreze
learning_rate = 0.001
nb_epochs = 1000
batch_size = 128

# Parametri arhitekture
nb_input = 12  # MNIST data input (img shape: 28*28)
nb_hidden1 = 256  # 1st layer number of neurons
nb_hidden2 = 256  # 2nd layer number of neurons
nb_classes = 2   # MNIST total classes (0-9 digits)

# Sama mreza
w = {
    '1': tf.Variable(tf.random.normal([nb_input, nb_hidden1], dtype=tf.float64)),
    '2': tf.Variable(tf.random.normal([nb_hidden1, nb_hidden2], dtype=tf.float64)),
    'out': tf.Variable(tf.random.normal([nb_hidden2, nb_classes], dtype=tf.float64))
}

b = {
    '1': tf.Variable(tf.random.normal([nb_hidden1], dtype=tf.float64)),
    '2': tf.Variable(tf.random.normal([nb_hidden2], dtype=tf.float64)),
    'out': tf.Variable(tf.random.normal([nb_classes], dtype=tf.float64))
}

# w = {
#     '1': tf.Variable(tf.keras.initializers.GlorotUniform()([nb_input, nb_hidden1], dtype=tf.float64)),
#     '2': tf.Variable(tf.keras.initializers.GlorotUniform()([nb_hidden1, nb_hidden2], dtype=tf.float64)),
#     'out': tf.Variable(tf.keras.initializers.GlorotUniform()([nb_hidden2, nb_classes], dtype=tf.float64))
# }

# b = {
#     '1': tf.Variable(tf.keras.initializers.GlorotUniform()([nb_hidden1], dtype=tf.float64)),
#     '2': tf.Variable(tf.keras.initializers.GlorotUniform()([nb_hidden2], dtype=tf.float64)),
#     'out': tf.Variable(tf.keras.initializers.GlorotUniform()([nb_classes], dtype=tf.float64))
# }

f = {
    '1': tf.nn.relu,
    '2': tf.nn.relu,
    'out': tf.nn.softmax
}

def runNN(x):
    z1 = tf.add(tf.matmul(x, w['1']), b['1'])
    a1 = f['1'](z1)
    z2 = tf.add(tf.matmul(a1, w['2']), b['2'])
    a2 = f['2'](z2)
    z_out = tf.add(tf.matmul(a2, w['out']), b['out']) # a2 ovde!
    out = f['out'](z_out)

    pred = tf.argmax(out, 1)

    return pred, z_out

# GD je djubre
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Trening!
for epoch in range(nb_epochs):
    epoch_loss = 0
    nb_batches = int(nb_train / batch_size)
    for i in range(nb_batches):
        x = train_x[i*batch_size : (i+1)*batch_size, :]
        y = train_y[i*batch_size : (i+1)*batch_size]
        y_onehot = tf.one_hot(y, nb_classes)

        with tf.GradientTape() as tape:
            _, z_out = runNN(x)

            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=z_out, labels=y_onehot))

        w1_g, w2_g, wout_g, b1_g, b2_g, bout_g = tape.gradient(loss, [w['1'], w['2'], w['out'], b['1'], b['2'], b['out']])

        opt.apply_gradients(zip([w1_g, w2_g, wout_g, b1_g, b2_g, bout_g], [w['1'], w['2'], w['out'], b['1'], b['2'], b['out']]))

        epoch_loss += loss

    # U svakoj epohi ispisujemo prosečan loss.
    epoch_loss /= nb_train
    print(f'Epoch: {epoch+1}/{nb_epochs}| Avg loss: {epoch_loss:.5f}')

# Test!
x = test_x
y = test_y

pred, _ = runNN(x)
pred_correct = tf.equal(pred, y)
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float32))

print(f'Test acc: {accuracy:.3f}')

Epoch: 1/1000| Avg loss: 0.80772
Epoch: 2/1000| Avg loss: 0.43200
Epoch: 3/1000| Avg loss: 0.37578
Epoch: 4/1000| Avg loss: 0.29827
Epoch: 5/1000| Avg loss: 0.23004
Epoch: 6/1000| Avg loss: 0.19858
Epoch: 7/1000| Avg loss: 0.16994
Epoch: 8/1000| Avg loss: 0.14711
Epoch: 9/1000| Avg loss: 0.12717
Epoch: 10/1000| Avg loss: 0.10868
Epoch: 11/1000| Avg loss: 0.09312
Epoch: 12/1000| Avg loss: 0.07894
Epoch: 13/1000| Avg loss: 0.06830
Epoch: 14/1000| Avg loss: 0.06072
Epoch: 15/1000| Avg loss: 0.05474
Epoch: 16/1000| Avg loss: 0.04969
Epoch: 17/1000| Avg loss: 0.04520
Epoch: 18/1000| Avg loss: 0.04119
Epoch: 19/1000| Avg loss: 0.03691
Epoch: 20/1000| Avg loss: 0.03400
Epoch: 21/1000| Avg loss: 0.03068
Epoch: 22/1000| Avg loss: 0.02818
Epoch: 23/1000| Avg loss: 0.02621
Epoch: 24/1000| Avg loss: 0.02395
Epoch: 25/1000| Avg loss: 0.02230
Epoch: 26/1000| Avg loss: 0.02057
Epoch: 27/1000| Avg loss: 0.01946
Epoch: 28/1000| Avg loss: 0.01825
Epoch: 29/1000| Avg loss: 0.01849
Epoch: 30/1000| Avg los